Please note this lab requires gym package <= 0.20. You can directly use pip install gym == 0.2 if you haven't installed gym package before.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm as _tqdm

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1)  # Safety, do not overflow buffer

%matplotlib inline

assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed!"

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import seaborn as sns
import pandas as pd

---
## 1. Policy Evaluation

In this exercise we will evaluate a policy, e.g. find the value function for a policy. The problem we consider is the gridworld from Example 4.1 in Sutton's Reinforcement Learning book. The environment is implemented as `GridworldEnv`, which is a subclass of the `Env` class from [OpenAI Gym](https://github.com/openai/gym). This means that we can interact with the environment. We can look at the documentation to see how we can interact with the environment.

In [ ]:
from gridworld import GridworldEnv
env = GridworldEnv()
# Lets see what this is
?env

In [ ]:
# To have a quick look into the code
??env

Now we want to evaluate a policy by using Dynamic Programming. For more information, see the [Intro to RL](http://www.incompleteideas.net/book/RLbook2020.pdf) book, section 4.1. This algorithm requires knowledge of the problem dynamics in the form of the transition probabilities $p(s',r|s,a)$. In general these are not available, but for our gridworld we know the dynamics and these can be accessed as `env.P`.

In [ ]:
# Take a moment to figure out what P represents. 
# Note that this is a deterministic environment. 
# What would a stochastic environment look like?
env.P[1]

In [ ]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        v = np.zeros(env.nS)
        for s in range(env.nS):
            for action, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.P[s][action]:
#                     print(prob, next_state, reward, done)
                    v[s]+= action_prob*prob*(reward+discount_factor*V[next_state])
        
            delta = max(delta, abs(v[s]-V[s]))
            V[s] = v[s]
        if delta < theta:
            break
        
    return np.array(V)


In [ ]:
#Your code here: Please initialize a random policy that would move to each direction with 1/4 probability
random_policy = np.ones([env.nS, env.nA]) / env.nA
#End of your code

In [ ]:
random_policy

In [ ]:
V = policy_eval(random_policy, env)
V

In [ ]:
def plot_gridworld_value(V):
    plt.figure()
    c = plt.pcolormesh(V, cmap='gray')
    plt.colorbar(c)
    plt.gca().invert_yaxis()  # In the array, first row = 0 is on top

# Making a plot always helps
plot_gridworld_value(V.reshape(env.shape))

---
## 2. Policy Iteration
Using the policy evaluation algorithm we can implement policy iteration to find a good policy for this problem. Note that we do not need to use a discount_factor for episodic tasks but make sure your implementation can handle this correctly!

In [ ]:
# calculate for each action in A, for state s
def backup(V, s, env, policy, discount_factor=1.0):
    V_a = np.zeros(env.nA)
    
    #NOTE: no expectation over actions, calculate for all actions in your Action space
        
    for action in range(env.nA) :
        policy[s,action] = 0
        for prob, next_state, reward, done in env.P[s][action]:
             V_a[action]+= prob*(reward+discount_factor*V[next_state])
    
    return V_a, policy

In [ ]:
def policy_improvement(env, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
#     print(np.arange(env.nA), policy[0])
#     np.random.choice(int(np.arange(env.nA)), policy[0])
    
    while True:
        
        V = policy_eval(policy, env, discount_factor)
        
        policy_stable = True
        for s in range(env.nS):
            
            #get an old action             
            #old_action = np.random.choice(np.arange(env.nA), 1, list(policy[s]))
            old_action = np.argmax(policy[s])
            
            #calculate V(s) for each s: by backup diagram (one-step lookahead)
            action_values, policy = backup(V, s, env, policy)
            #print(policy)
            
            a_max_i = np.argwhere(action_values == np.amax(action_values))
            policy[s,a_max_i.flatten()] = 1
            policy[s,:] = policy[s,:]/np.sum([policy[s,:]])
             
            if np.argmax(policy[s,:])!=old_action:
                policy_stable = False
       
        
        if policy_stable:
            break 
    
    return policy, V

In [ ]:
# Let's see what it does
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

def print_grid_policy(policy, symbols=["^", ">", "v", "<"]):
    symbols = np.array(symbols)
    for row in policy:
        print("".join(symbols[row]))

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print_grid_policy(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

plot_gridworld_value(v.reshape(env.shape))

---
## 3. Value Iteration
Now implement the value iteration algorithm.

In [ ]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    

    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    
    # Implement!
    while True:
        delta = 0
        for s in range(env.nS):
            
            # Get action_values and best_val
            # YOUR CODE HERE
            action_values,_ = backup(V, s, env, policy) 
            best_val = np.max(action_values)
            # End of code
            
            delta = max(delta, abs(best_val-V[s]))
            V[s] = best_val
            
        if delta < theta:
            break
            
    #calculate optimal policy
    for s in range(env.nS):
        action_values, policy = backup(V, s, env, policy)
        
        a_max_i = np.argwhere(action_values == np.amax(action_values))
        policy[s,a_max_i.flatten()] = 1
        policy[s,:] = policy[s,:]/np.sum([policy[s,:]])
      
    return policy, V

In [ ]:
# Oh let's test again
# Let's see what it does
policy, v = value_iteration(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print_grid_policy(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")


What is the difference between value iteration and policy iteration? Which algorithm is most efficient (e.g. needs to perform the least *backup* operations)? Please answer *concisely* in the cell below.

**YOUR ANSWER HERE**





Two popular algorithms for solving Markov Decision Processes (MDPs) and obtaining the optimal policy are value iteration and policy iteration. The key difference between them lies in how they update the policy and the value function.

With value iteration, an initial value function is used and iteratively updated until it converges to the optimal value function. At each iteration, the value of each state-action pair is computed and used to update the value function. The optimal policy is then computed based on the updated value function.

Policy iteration, on the other hand, starts with an initial policy and gradually improves it until it converges to the optimal policy. At each iteration, the policy is evaluated to obtain the value function, and the policy is then updated based on the value function.

Although policy iteration converges faster than value iteration due to updating both the policy and value function, it requires more computation per iteration, which means that the total number of operations may be higher for policy iteration.